In [ ]:
print("hello world")

In [4]:
import keras

In [5]:
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten,Dense,GlobalAveragePooling2D,Lambda,add, Dropout, Activation,Concatenate,BatchNormalization
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras import backend as K
import matplotlib.pyplot as plt
import cv2

In [6]:
import scipy
from scipy.spatial import distance

In [7]:
from imageio import imread
from skimage.transform import resize

In [8]:
import os
from os import listdir
from os.path import isfile, join

In [9]:
import mtcnn
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN
model2 = MTCNN(min_face_size = 10)

In [10]:
from inception_resnet_v1 import *

In [11]:
model = InceptionResNetV1()

In [12]:
model.load_weights('facenet_keras_weights.h5')

In [13]:
def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [14]:
def preprocess(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

In [15]:
def faceinfo(image,margin =10):
    person_image1 = cv2.imread(image)
    person_image = cv2.cvtColor(person_image1, cv2.COLOR_BGR2RGB)
    face_info = model2.detect_faces(person_image)
    for face in face_info:
        x,y,w,h = face['box']
        x,y = abs(x),abs(y)
        face = person_image[y-margin:y+h+margin,x-margin:x+w+margin]
        roi = cv2.resize(face, (160,160))
        img = np.expand_dims(roi, axis=0)
    return img

In [16]:
def faceinfo2(image,margin =10):
    person_image1 = cv2.imread(image)
    person_image = cv2.cvtColor(person_image1, cv2.COLOR_BGR2RGB)
    face_info = model2.detect_faces(person_image)
    for face in face_info:
        if face['confidence']>0.98:
            x,y,w,h = face['box']
            x,y = abs(x),abs(y)
            face = person_image1[y-margin//2:y+h+margin//2,x-margin//2:x+w+margin//2,:]
            roi = cv2.resize(face, (160,160))
            break
        else:
            pass
    return roi

In [17]:
def expand(image):
    person_image1 = cv2.imread(image)
    person_image = cv2.cvtColor(person_image1, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(person_image, axis=0)
    return img

In [ ]:
epsilon = 0.82
def Face_match(img1, img2):
    img1_representation = model.predict(preprocess(faceinfo(img1)))
    img2_representation = model.predict(preprocess(faceinfo(img2)))
    img1_representation = l2_normalize(img1_representation)
    img2_representation = l2_normalize(img2_representation)
    
    e_distance = distance.euclidean(img1_representation, img2_representation)
    
    f = plt.figure(figsize = (10,6))
    f.add_subplot(1,2, 1)
    image1 = plt.imread(img1)
    plt.imshow(image1)
    plt.xticks([]); plt.yticks([])
    f.add_subplot(1,2, 2)
    image2 = plt.imread(img2)
    plt.imshow(image2)
    plt.xticks([]); plt.yticks([])
    plt.show(block=True)
    
    print("euclidean_distance: ",e_distance)
    
    if(e_distance < epsilon):
        print("They are same person")
    else:
        print("They are not same person!")

In [ ]:
import time
start = time.time()
Face_match("Bill_Gates1.jpg", "Larry_Page2.jpg")
end = time.time()
diff = end-start
print(diff)

In [ ]:
import time
start = time.time()
Face_match("kiara_advani2.jpg", "Kiara_Advani3.jpg")
end = time.time()
diff = end-start
print(diff)

In [ ]:
l = listdir('faces3')

In [ ]:
im = faceinfo2('Saif-Ali-Khan5.jpg')
cv2.imwrite('crop_faces2/Saif_Ali_Khan5.jpg',im)

In [ ]:
def add_photo(image_path):
    name = image_path.split(".")[0].split('/')[1]
    im = faceinfo2(image_path)
    cv2.imwrite('crop_faces2/'+name+ '.jpg',im)
    im2 = np.expand_dims(im, axis=0)
    im3 = l2_normalize(model.predict(preprocess(im2)))
    all_people_faces[name] = im3

In [ ]:
l = listdir('faces3')
len(l)

In [ ]:
m =0
for i in l:
    add_photo('faces3/'+i)
    print(m)
    m=m+1

In [ ]:
i = 1
pictures = "./crop_faces2/"
all_people_faces = dict()
for face in listdir(pictures):
    person_face = face.split(".")[0]
    all_people_faces[person_face] = l2_normalize(model.predict(preprocess(expand(pictures+face))))
    print(i)
    i = i+1

In [1]:
import pickle

In [ ]:
with open('all_people_faces' + '.pkl', 'wb') as f:
    pickle.dump(all_people_faces, f, pickle.HIGHEST_PROTOCOL)

In [3]:
all_people_faces = pickle.load(open('all_people_faces.pkl','rb'))

In [20]:
import time

In [21]:
def match_face(image_address,margin = 5):
    t1 = time.time()
    image = cv2.imread(image_address)
    image1 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)    
    face_info = model2.detect_faces(image1)
    for face in face_info:
        if face['confidence']>0.98:
            x,y,w,h = face['box']
            x,y = abs(x),abs(y)
            if x <10: 
                x = 10
            elif y< 10:
                y = 10
            face = image1[y-margin//2:y+h+margin//2,x-margin//2:x+w+margin//2,:]
            roi =  resize(face, (160,160),mode = 'reflect')
            roi = np.expand_dims(roi, axis=0)
            captured_representation = l2_normalize(model.predict(preprocess(roi))[0])
            cv2.rectangle(image, (x,y), (x+w,y+h), (0,0,255), 1)
            cv2.rectangle(image,(x,y-20),(x+w+40,y),(0,255,0),-1)
            sim_dict = {}
            for i in all_people_faces:
                person_name = i
                representation = all_people_faces[i]
                similarity =  distance.euclidean(representation, captured_representation)
                sim_dict[person_name] = similarity
                min_similarity = min(sim_dict.values())
            found = 0
            if min_similarity < 0.75:
                person_name1 = [key for key in sim_dict if sim_dict[key] == min_similarity]
                cv2.putText(image, person_name1[0][:-1], (int(x), int(y-5)), 0, 0.6, (0, 0, 255), 2)
                found = 1
                print(person_name1[0][:-1])
                print(min_similarity)
            if(found == 0):
                cv2.putText(image, 'unknown', (int(x), int(y-5)), 0, 0.6, (255, 0, 0), 2)
            
        else:
            pass
    t2 = time.time()
    print('time took ' + str(t2-t1) + ' sec')
    cv2.imshow('Face Recognition', image)
    cv2.waitKey(0)    
    cv2.destroyAllWindows()

In [28]:
match_face('Test Images2/70.jpeg')

time took 1.223818063735962 sec


In [ ]:
def match_face2(image_address,margin = 10):
    image = image_address
    image1 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)    
    face_info = model2.detect_faces(image1)
    for face in face_info:
        if face['confidence']>0.98:
            x,y,w,h = face['box']
            x,y = abs(x),abs(y)
            if x <10: 
                x = 10
            elif y< 10:
                y = 10
            face = image1[y-margin//2:y+h+margin//2,x-margin//2:x+w+margin//2,:]
            roi =  resize(face, (160,160),mode = 'reflect')
            roi = np.expand_dims(roi, axis=0)
            captured_representation = l2_normalize(model.predict(preprocess(roi))[0])
            
        
            cv2.rectangle(image, (x,y), (x+w,y+h), (0,0,255), 1)
            cv2.rectangle(image,(x,y-20),(x+w+40,y),(0,255,0),-1)
            sim_dict = {}
            for i in all_people_faces:
                person_name = i
                representation = all_people_faces[i]

                similarity =  distance.euclidean(representation, captured_representation)
                sim_dict[person_name] = similarity
                min_similarity = min(sim_dict.values())
            
            found = 0
            if min_similarity < 0.82:
                person_name1 = [key for key in sim_dict if sim_dict[key] == min_similarity]
                cv2.putText(image, person_name1[0][:-1], (int(x), int(y-5)), 0, 0.6, (0, 0, 255), 2)
                found = 1
                
                
            if(found == 0):
                cv2.putText(image, 'unknown', (int(x), int(y-5)), 0, 0.6, (255, 0, 0), 2)
            
        else:
            pass
    
    cv2.imshow('Face Recognition', image)

In [ ]:
import time

In [ ]:
start_time = time.time()
cap = cv2.VideoCapture('test_video1.mp4')
while True:
    now_time = time.time()
    if now_time -start_time >.2:
        ret, img = cap.read()
        if not ret:
            break
        match_face2(img)
        start_time = time.time()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
cap.release()
cv2.destroyAllWindows()

In [ ]:
start_time = time.time()
cap = cv2.VideoCapture('test_video2.mp4')
i = 0
while True:
    now_time = time.time()
    if now_time -start_time >.02:
        ret, img = cap.read()
        if not ret:
            break
        if i%30 ==0:
            match_face2(img)
            
        start_time = time.time()
        i = i+1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
cap.release()
cv2.destroyAllWindows()

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
model = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
mtcnn = MTCNN(image_size= 160, margin=10,keep_all = True)
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__